### Library and data needed

In [128]:
import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [129]:
opts_k = 10
np.random.seed(0)


### Preprocessing : Read file and prepare input 

In [130]:
# reading file
os.chdir('../backend')
file_path = os.getcwd() + "/metadata.csv"
df = pd.read_csv(file_path)
column_names = df.columns

# Filter columns starting with 'feature_' as x value and 'algo_' as y value
x_columns = [col for col in column_names if col.startswith('feature_')]
y_columns = [col for col in column_names if col.startswith('algo_')]


# Extract x and y from selected columns 
x = df[x_columns]
y = df[y_columns]

# Skip feature selection if <= 3 features and all features are kept
if x.shape[1] <= 3:
    print("There are 3 or less features to do selection. Skipping feature selection.")
    # TODO: return the desire output format
elif x.shape[1] <= 1:
    print("There is only 1 feature. Stopping space construction.")
    # TODO: return the desire output format
    
n_feature = x.shape[1]

In [131]:
x.head()

,feature_Max_Normalized_Entropy_attributes,feature_Normalized_Entropy_Class_Attribute,feature_Mean_Mutual_Information_Attribute_Class,feature_ErrorRate_Decision_Node,feature_WeightedDist_StdDev,feature_Max_Feature_Efficiency_F3,feature_Collective_Feature_Efficiency_F4,feature_Training_Error_Linear_Classifier_L2,feature_Fraction_Points_Class_Boundary_N1,feature_Nonlinearity_Nearest_Neighbor_Classifier_N4
0,0.332548,0.131687,0.229928,0.386459,2.343235,0.076929,0.095204,0.260235,0.404836,0.327388
1,0.332548,0.299462,0.368704,0.210410,2.343235,0.500693,0.555474,0.035714,0.082629,0.357113
2,0.297231,0.106740,0.105274,0.141818,0.745009,0.292453,0.754717,0.198113,0.292453,0.212264
3,0.313965,0.202552,0.146191,0.219275,0.814613,0.134146,0.134146,0.154472,0.357724,0.288618
4,0.279463,0.437579,0.100076,0.240355,0.690097,0.827618,0.966261,0.031711,0.041482,0.030697


In [132]:
y.head()

,algo_NB,algo_LDA,algo_QDA,algo_CART,algo_J48,algo_KNN,algo_L_SVM,algo_poly_SVM,algo_RBF_SVM,algo_RandF
0,0.280096,0.239715,0.254012,0.245339,0.256491,0.256969,0.230584,0.277411,0.221198,0.234432
1,0.164182,0.159577,0.895472,0.172975,0.165282,0.167113,0.171518,0.178773,0.169919,0.895472
2,0.141818,0.123636,0.170000,0.162727,0.132727,0.178182,0.123636,0.123636,0.132727,0.132727
3,0.253286,0.264329,0.554844,0.264329,0.264329,0.246951,0.264329,0.229573,0.231402,0.244275
4,0.034967,0.859129,0.859129,0.060208,0.038009,0.072298,0.035244,0.265880,0.151676,0.859129


### Features Selection (Correlation Based): Line 39-53)


In [133]:
# Initialize empty matrices for correlation coefficients and p-values
corr_matrix = pd.DataFrame(index=x.columns, columns=y.columns)
p_value_matrix = pd.DataFrame(index=x.columns, columns=y.columns)

# Compute correlation coefficient and p-value for each pair of variables
for x_col in x.columns:
    for y_col in y.columns:
        corr_coef, p_value = pearsonr(x[x_col], y[y_col])
        corr_matrix.loc[x_col, y_col] = corr_coef
        p_value_matrix.loc[x_col, y_col] = p_value

# handle nan value, p-value greater than 0.5 and also absolute value for corr_matrix
p_value_matrix[p_value_matrix > 0.05] = 0
corr_matrix = abs(corr_matrix)

In [134]:
corr_matrix

,algo_NB,algo_LDA,algo_QDA,algo_CART,algo_J48,algo_KNN,algo_L_SVM,algo_poly_SVM,algo_RBF_SVM,algo_RandF
feature_Max_Normalized_Entropy_attributes,0.155569,0.184577,0.083635,0.284112,0.139638,0.139925,0.262739,0.241402,0.288322,0.206101
feature_Normalized_Entropy_Class_Attribute,0.181881,0.419038,0.698115,0.044732,0.00367,0.023107,0.026912,0.067973,0.098525,0.308929
feature_Mean_Mutual_Information_Attribute_Class,0.469024,0.020967,0.289984,0.38408,0.411127,0.446141,0.378365,0.380629,0.372192,0.149177
feature_ErrorRate_Decision_Node,0.533626,0.344297,0.261357,0.598178,0.615601,0.585527,0.586799,0.689633,0.544109,0.308655
feature_WeightedDist_StdDev,0.24205,0.23178,0.203043,0.349398,0.349154,0.3307,0.298319,0.386479,0.342066,0.184245
feature_Max_Feature_Efficiency_F3,0.498631,0.023974,0.155385,0.469903,0.504981,0.482735,0.41571,0.41463,0.37701,0.033425
feature_Collective_Feature_Efficiency_F4,0.270725,0.005447,0.088412,0.366902,0.371975,0.382477,0.334577,0.361583,0.278572,0.086481
feature_Training_Error_Linear_Classifier_L2,0.627241,0.225743,0.103778,0.668658,0.705683,0.651858,0.686151,0.640647,0.643098,0.259195
feature_Fraction_Points_Class_Boundary_N1,0.732773,0.269194,0.027883,0.844128,0.860762,0.911544,0.750021,0.728602,0.764341,0.356168
feature_Nonlinearity_Nearest_Neighbor_Classifier_N4,0.44207,0.000364,0.129661,0.397288,0.412983,0.374073,0.377263,0.25912,0.327465,0.148677


In [135]:
p_value_matrix

,algo_NB,algo_LDA,algo_QDA,algo_CART,algo_J48,algo_KNN,algo_L_SVM,algo_poly_SVM,algo_RBF_SVM,algo_RandF
feature_Max_Normalized_Entropy_attributes,0.02348,0.007044,0,0.000027,0.042245,0.041819,0.000108,0.00039,0.00002,0.002565
feature_Normalized_Entropy_Class_Attribute,0.007937,0.0,0.0,0,0,0,0,0,0,0.000005
feature_Mean_Mutual_Information_Attribute_Class,0.0,0,0.000018,0.0,0.0,0.0,0.0,0.0,0.0,0.029903
feature_ErrorRate_Decision_Node,0.0,0.0,0.000118,0.0,0.0,0.0,0.0,0.0,0.0,0.000005
feature_WeightedDist_StdDev,0.000376,0.000671,0.002979,0.0,0.0,0.000001,0.00001,0.0,0.0,0.007148
feature_Max_Feature_Efficiency_F3,0.0,0,0.023646,0.0,0.0,0.0,0.0,0.0,0.0,0
feature_Collective_Feature_Efficiency_F4,0.000065,0,0,0.0,0.0,0.0,0.000001,0.0,0.000039,0
feature_Training_Error_Linear_Classifier_L2,0.0,0.000932,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000135
feature_Fraction_Points_Class_Boundary_N1,0.0,0.000072,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
feature_Nonlinearity_Nearest_Neighbor_Classifier_N4,0.0,0,0,0.0,0.0,0.0,0.0,0.000136,0.000001,0.030463


In [136]:
# TODO: 
# Initial approach: 
# 1. sort features based on algorithm
# 2. For each algorithm, take the highest feature, then pick feature > rho in json
# 3. Only drop feature if feature has correlation value < rho in every algorithms
# Current Approach : for each feature, take mean of them, dropping feature with correlation value < rho in json
# why p-value matrix is not used after dropping p > 0.5?
for index, row in corr_matrix.iterrows():
    row_mean = row.mean()
    # 0.1 value is taken from options.json['sifted']['rho']
    if row_mean < 0.1:
        corr_matrix.drop(index, inplace=True)
print(f'After correlation-based selection, the number of features selected is {corr_matrix.shape[0]}')
# Filter the x matrix that only contain the feature selected
x_aux = x[corr_matrix.index]
# TODO: line 55-67: if nfeatures <=1, <= 3, <= opts.K

After correlation-based selection, the number of features selected is 10


In [137]:
x_aux

,feature_Max_Normalized_Entropy_attributes,feature_Normalized_Entropy_Class_Attribute,feature_Mean_Mutual_Information_Attribute_Class,feature_ErrorRate_Decision_Node,feature_WeightedDist_StdDev,feature_Max_Feature_Efficiency_F3,feature_Collective_Feature_Efficiency_F4,feature_Training_Error_Linear_Classifier_L2,feature_Fraction_Points_Class_Boundary_N1,feature_Nonlinearity_Nearest_Neighbor_Classifier_N4
0,0.332548,0.131687,0.229928,0.386459,2.343235,0.076929,0.095204,0.260235,0.404836,0.327388
1,0.332548,0.299462,0.368704,0.210410,2.343235,0.500693,0.555474,0.035714,0.082629,0.357113
2,0.297231,0.106740,0.105274,0.141818,0.745009,0.292453,0.754717,0.198113,0.292453,0.212264
3,0.313965,0.202552,0.146191,0.219275,0.814613,0.134146,0.134146,0.154472,0.357724,0.288618
4,0.279463,0.437579,0.100076,0.240355,0.690097,0.827618,0.966261,0.031711,0.041482,0.030697
...,...,...,...,...,...,...,...,...,...,...
207,0.329152,0.145087,0.046484,0.374157,5.507321,0.150201,0.231316,0.142197,0.176499,0.402087
208,0.305765,0.099196,0.352238,0.090772,3.618310,0.121523,0.243045,0.032211,0.060029,0.038067
209,0.304303,0.103632,0.015105,0.237895,0.812286,0.176768,0.989899,0.237374,0.424242,0.244950
210,0.305485,0.103976,0.014623,0.236053,0.812286,0.180412,0.994845,0.237113,0.422680,0.247423


### K-means Clustering (Line 69-95)

- Applies k-means clustering algorithm to Xaux
- Evaluates sihouette values for different number of clusters
- Output -> (1) inspected num of clusters, (2) corresponding criterion values
- Checks, if sihouette value for specified number of clusters is below 0.5 (sihouette value is  a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation), the range is -1 to 1)
    - we do average sihouette value for each cluster
    - if ASV is < 0.5 ==> poor cluster quality and suggest increasing opts.K. Or it will find a value of K that yields sihouette value above 7.5, if available
- After checking sihouette value, do k-mean clustering

***Steps***
1. construct kmc and compute silhouette_score for chosen k clusters
2. repeat step 1 for k clusters value ranging from 3 to n_feature
3. display average of silhouette score based on k clusters
3. report if your chosen k-mean based on options.json is lower than 0.5
    - if lower than 0.5, based on step 2, find the best k clusters that has silhouette score higher than 7.5

***k clusters***
- using k value from options.json, do kmc

In [138]:
np.random.seed(0)

# Initialize empty dictionary to store silhouette scores
silhouette_scores = {}

# Range of cluster numbers to evaluate
min_clusters = 3
max_clusters = n_feature  # Assuming nfeats is defined
cluster_range = range(min_clusters, max_clusters + 1)

# # Evaluating silhouette scores for different numbers of clusters
for n_clusters in cluster_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(x_aux)  # Transpose Xaux for clustering
    silhouette_scores[n_clusters] = silhouette_score(x_aux, cluster_labels, metric='correlation')

# Displaying average silhouette values for each number of clusters
print('-> Average silhouette values for each number of clusters.')
for n_clusters, silhouette_score in silhouette_scores.items():
    print(f"Number of clusters: {n_clusters}, Silhouette score: {silhouette_score}")

# Checking silhouette value for opts.K
if silhouette_scores[opts_k] < 0.5:
    print(f'-> The silhouette value for K={opts_k} is below 0.5. You should consider increasing K.')
    K_suggested = next((k for k, score in silhouette_scores.items() if score > 0.75), None)
    if K_suggested is not None:
        print(f'A suggested value of K is {K_suggested}.')
    else:
        print(f'No suggested value of K')
# TODO: silhouette score is way different, perhaps the x_aux is very different

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.1

-> Average silhouette values for each number of clusters.
Number of clusters: 3, Silhouette score: -0.12405225017366835
Number of clusters: 4, Silhouette score: -0.2904688623794731
Number of clusters: 5, Silhouette score: -0.037007130208502756
Number of clusters: 6, Silhouette score: 0.003774497919011957
Number of clusters: 7, Silhouette score: -0.010491846091226302
Number of clusters: 8, Silhouette score: -0.07267992537168953
Number of clusters: 9, Silhouette score: -0.060270225666678424
Number of clusters: 10, Silhouette score: -0.06455660072222497
-> The silhouette value for K=10 is below 0.5. You should consider increasing K.
No suggested value of K


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
